In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# ✅ Step 1: Create a fresh project folder structure on Google Drive
import os

base_dir = "/content/drive/MyDrive/Cancer_WES_Project"
subdirs = [
    "raw_fastq",
    "reference",
    "bam_files",
    "annotation",
    "vcf_files",
    "results"
]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

print("✅ Project folder structure created successfully.")


✅ Project folder structure created successfully.


In [3]:
# ✅ Install core bioinformatics tools
!apt-get update -qq
!apt-get install -y -qq fastqc bwa samtools bcftools bedtools
!wget -q https://github.com/broadinstitute/gatk/releases/download/4.4.0.0/gatk-4.4.0.0.zip
!unzip -q gatk-4.4.0.0.zip
!mv gatk-4.4.0.0 /opt/gatk
!chmod +x /opt/gatk/gatk


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../00-libhtscodecs2_1.1.1-3_amd64.deb ...
Unpacking libhtscodecs2:amd64 (1.1.1-3) ...
Selecting previously unselected package libhts3:amd64.
Preparing to unpack .../01-libhts3_1.13+ds-2build1_amd64.deb ...
Unpacking libhts3:amd64 (1.13+ds-2build1) ...
Selecting previously unselected package bcftools.
Preparing to unpack .../02-bcftools_1.13-1_amd64.deb ...
Unpacking bcftools (1.13-1) ...
Selecting previously unselected package bedtools.
Preparing to unpack .../03-bedtools_2.30.0+dfsg-2ubuntu0.1_amd64.deb ...
Unpacking bedtools (2.30.0+dfsg-2ubuntu0.1) ...
Selecting previously unselected package bwa.
Preparin

In [4]:
!fastqc --version
!bwa 2>&1 | head -n 1
!samtools --version | head -n 1
!/opt/gatk/gatk --version
!bcftools --version | head -n 1


FastQC v0.11.9

samtools 1.13
Using GATK jar /opt/gatk/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/gatk/gatk-package-4.4.0.0-local.jar --version
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0
bcftools 1.13


In [5]:
# Remove older Java versions if needed
!sudo apt-get remove openjdk-11-jre-headless -y
!sudo apt-get update

# Install Java 17
!sudo apt install openjdk-17-jre-headless -y

# Verify version
!java -version


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  fonts-dejavu-extra java-common libapache-pom-java libargs4j-java
  libatk-wrapper-java libatk-wrapper-java-jni libcommons-compress-java
  libcommons-io-java libcommons-jexl2-java libcommons-lang3-java
  libcommons-logging-java libcommons-math3-java libcommons-parent-java
  libfindbin-libs-perl libhtsjdk-java libjbzip2-java libjson-simple-java
  libngs-java libngs-sdk-dev libngs-sdk2 libpcsclite1 libsis-base-java
  libsis-base-jni libsis-jhdf5-java libsis-jhdf5-jni libsnappy-java
  libsnappy-jni
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  ca-certificates-java default-jre default-jre-headless fastqc
  openjdk-11-jdk-headless openjdk-11-jre openjdk-11-jre-headless
0 upgraded, 0 newly installed, 7 to remove and 35 not upgraded.
After this operation, 260 MB disk space will be

In [6]:
!/opt/gatk/gatk --version


Using GATK jar /opt/gatk/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/gatk/gatk-package-4.4.0.0-local.jar --version
The Genome Analysis Toolkit (GATK) v4.4.0.0
HTSJDK Version: 3.0.5
Picard Version: 3.0.0


In [7]:
# ✅ Install SRA Toolkit
!apt-get install -y sra-toolkit

# ✅ Confirm installation
!which prefetch
!which fasterq-dump


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  fonts-dejavu-extra libapache-pom-java libargs4j-java libatk-wrapper-java
  libatk-wrapper-java-jni libcommons-compress-java libcommons-io-java
  libcommons-jexl2-java libcommons-lang3-java libcommons-logging-java
  libcommons-math3-java libcommons-parent-java libfindbin-libs-perl
  libhtsjdk-java libjbzip2-java libjson-simple-java libngs-java libngs-sdk-dev
  libngs-sdk2 libsis-base-java libsis-base-jni libsis-jhdf5-java
  libsis-jhdf5-jni libsnappy-java libsnappy-jni
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  blends-common libkdf5-2 libncbi-vdb2 libncbi-wvdb2 med-config menu
Suggested packages:
  blends-doc menu-l10n gksu | kde-runtime | ktsuss
The following NEW packages will be installed:
  blends-common libkdf5-2 libncbi-vdb2 libncbi-wvdb2 med-config menu
  s

In [8]:
# ✅ Step: Go to the FASTQ folder
%cd /content/drive/MyDrive/Cancer_WES_Project/raw_fastq

# ✅ Step: Extract FASTQ with 4 threads
!fasterq-dump SRR766039  -e 4

# ✅ Step: Compress the FASTQ files
!gzip SRR766039_1.fastq
!gzip SRR766039_2.fastq


/content/drive/MyDrive/Cancer_WES_Project/raw_fastq
spots read      : 32,182,906
reads read      : 64,365,812
reads written   : 64,365,812


In [9]:
# ✅ Reinstall FastQC
!apt-get update -qq
!apt-get install -y fastqc
!fastqc --version


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  default-jre default-jre-headless openjdk-11-jre openjdk-11-jre-headless
Suggested packages:
  libnss-mdns fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  default-jre default-jre-headless fastqc openjdk-11-jre
  openjdk-11-jre-headless
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 43.2 MB of archives.
After this operation, 177 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 openjdk-11-jre-headless amd64 11.0.28+6-1ubuntu1~22.04.1 [42.6 MB]
Get:2 http://archive.ubuntu.com/ub

In [10]:
# ✅ Move into the FASTQ folder
%cd /content/drive/MyDrive/Cancer_WES_Project/raw_fastq

# ✅ Run FastQC on both files
!fastqc SRR766039_1.fastq.gz SRR766039_2.fastq.gz


/content/drive/MyDrive/Cancer_WES_Project/raw_fastq
Started analysis of SRR766039_1.fastq.gz
Approx 5% complete for SRR766039_1.fastq.gz
Approx 10% complete for SRR766039_1.fastq.gz
Approx 15% complete for SRR766039_1.fastq.gz
Approx 20% complete for SRR766039_1.fastq.gz
Approx 25% complete for SRR766039_1.fastq.gz
Approx 30% complete for SRR766039_1.fastq.gz
Approx 35% complete for SRR766039_1.fastq.gz
Approx 40% complete for SRR766039_1.fastq.gz
Approx 45% complete for SRR766039_1.fastq.gz
Approx 50% complete for SRR766039_1.fastq.gz
Approx 55% complete for SRR766039_1.fastq.gz
Approx 60% complete for SRR766039_1.fastq.gz
Approx 65% complete for SRR766039_1.fastq.gz
Approx 70% complete for SRR766039_1.fastq.gz
Approx 75% complete for SRR766039_1.fastq.gz
Approx 80% complete for SRR766039_1.fastq.gz
Approx 85% complete for SRR766039_1.fastq.gz
Approx 90% complete for SRR766039_1.fastq.gz
Approx 95% complete for SRR766039_1.fastq.gz
Analysis complete for SRR766039_1.fastq.gz
Started an

In [12]:
from google.colab import files

# Download both reports
files.download("/content/drive/MyDrive/Cancer_WES_Project/raw_fastq/SRR766039_1_fastqc.html")
files.download("/content/drive/MyDrive/Cancer_WES_Project/raw_fastq/SRR766039_2_fastqc.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# Step 0: Delete old corrupted files (if any)
!rm -f /content/reference/GRCh38_no_alt_plus_decoy.fa*
!mkdir -p /content/reference
%cd /content/reference

# Step 1: Redownload cleanly (~834 MB compressed)
!wget -O GRCh38_no_alt_plus_decoy.fa.gz "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GRCh38_major_release_seqs_for_alignment_pipelines/GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fna.gz"

# Step 2: Decompress (~10–11 GB uncompressed)
!gunzip -f GRCh38_no_alt_plus_decoy.fa.gz

# Step 3: Confirm proper decompressed size
!ls -lh /content/reference/GRCh38_no_alt_plus_decoy.fa


/content/reference
--2025-08-07 07:52:18--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GRCh38_major_release_seqs_for_alignment_pipelines/GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.7, 130.14.250.10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874637925 (834M) [application/x-gzip]
Saving to: ‘GRCh38_no_alt_plus_decoy.fa.gz’

GRCh38_no_alt_plus_ 100%[===================>] 834.12M  45.2MB/s    in 17s     

2025-08-07 07:52:35 (49.2 MB/s) - ‘GRCh38_no_alt_plus_decoy.fa.gz’ saved [874637925/874637925]

-rw-r--r-- 1 root root 3.0G Feb 17  2016 /content/reference/GRCh38_no_alt_plus_decoy.fa


In [14]:
# Count number of lines
!wc -l /content/reference/GRCh38_no_alt_plus_decoy.fa

# Show how many sequences (chromosomes/contigs) are in the file
!grep "^>" /content/reference/GRCh38_no_alt_plus_decoy.fa | wc -l

# Print the first 10 sequence headers
!grep "^>" /content/reference/GRCh38_no_alt_plus_decoy.fa | head -10


44371205 /content/reference/GRCh38_no_alt_plus_decoy.fa
2580
>chr1  AC:CM000663.2  gi:568336023  LN:248956422  rl:Chromosome  M5:6aef897c3d6ff0c78aff06ac189178dd  AS:GRCh38
>chr2  AC:CM000664.2  gi:568336022  LN:242193529  rl:Chromosome  M5:f98db672eb0993dcfdabafe2a882905c  AS:GRCh38
>chr3  AC:CM000665.2  gi:568336021  LN:198295559  rl:Chromosome  M5:76635a41ea913a405ded820447d067b0  AS:GRCh38
>chr4  AC:CM000666.2  gi:568336020  LN:190214555  rl:Chromosome  M5:3210fecf1eb92d5489da4346b3fddc6e  AS:GRCh38
>chr5  AC:CM000667.2  gi:568336019  LN:181538259  rl:Chromosome  M5:a811b3dc9fe66af729dc0dddf7fa4f13  AS:GRCh38  hm:47309185-49591369
>chr6  AC:CM000668.2  gi:568336018  LN:170805979  rl:Chromosome  M5:5691468a67c7e7a7b5f2a3a683792c29  AS:GRCh38
>chr7  AC:CM000669.2  gi:568336017  LN:159345973  rl:Chromosome  M5:cc044cc2256a1141212660fb07b6171e  AS:GRCh38
>chr8  AC:CM000670.2  gi:568336016  LN:145138636  rl:Chromosome  M5:c67955b5f7815a9a1edfaa15893d3616  AS:GRCh38
>chr9  AC:CM000671.2 

In [15]:
# ✅ Move to reference folder (just in case)
%cd /content/reference

# ✅ Create FASTA index
!samtools faidx GRCh38_no_alt_plus_decoy.fa

# ✅ Create BWA index
!bwa index GRCh38_no_alt_plus_decoy.fa




/content/reference
[bwa_index] Pack FASTA... 30.44 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6211430126, availableWord=449059136
[BWTIncConstructFromPacked] 10 iterations done. 99999998 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999998 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999998 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999998 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999998 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999998 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999998 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999998 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999998 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999998 characters processed.
[BWTIncConstructFromPacked] 11

In [16]:
# ✅ Install GATK (Genome Analysis Toolkit)
!wget -q https://github.com/broadinstitute/gatk/releases/download/4.4.0.0/gatk-4.4.0.0.zip
!unzip -q gatk-4.4.0.0.zip
!mv gatk-4.4.0.0 /opt/gatk
!ln -s /opt/gatk/gatk /usr/local/bin/gatk
!gatk --version


Using GATK jar /opt/gatk/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/gatk/gatk-package-4.4.0.0-local.jar --version
The Genome Analysis Toolkit (GATK) v4.4.0.0
HTSJDK Version: 3.0.5
Picard Version: 3.0.0


In [17]:
# ✅ Create GATK sequence dictionary
!gatk CreateSequenceDictionary \
   -R GRCh38_no_alt_plus_decoy.fa \
   -O GRCh38_no_alt_plus_decoy.dict

Using GATK jar /opt/gatk/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/gatk/gatk-package-4.4.0.0-local.jar CreateSequenceDictionary -R GRCh38_no_alt_plus_decoy.fa -O GRCh38_no_alt_plus_decoy.dict
09:48:36.706 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/gatk/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Thu Aug 07 09:48:36 UTC 2025] CreateSequenceDictionary --OUTPUT GRCh38_no_alt_plus_decoy.dict --REFERENCE GRCh38_no_alt_plus_decoy.fa --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUENCES 2147483647 --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false
[Thu Aug 07 0

In [18]:
# ✅ Final, clean alignment to local reference
sample_id = "SRR766039"

fq1 = f"/content/drive/MyDrive/Cancer_WES_Project/raw_fastq/{sample_id}_1.fastq.gz"
fq2 = f"/content/drive/MyDrive/Cancer_WES_Project/raw_fastq/{sample_id}_2.fastq.gz"
ref = "/content/reference/GRCh38_no_alt_plus_decoy.fa"
bam_dir = "/content/drive/MyDrive/Cancer_WES_Project/bam_files"

raw_bam = f"{bam_dir}/{sample_id}_raw.bam"
sorted_bam = f"{bam_dir}/{sample_id}_sorted.bam"
rg_bam = f"{bam_dir}/{sample_id}_RG.bam"

!mkdir -p {bam_dir}

# ✅ Step 1: BWA MEM
!bwa mem -t 2 {ref} {fq1} {fq2} | samtools view -Sb - > {raw_bam}

# ✅ Step 2: Sort
!samtools sort -o {sorted_bam} {raw_bam}

# ✅ Step 3: Add read groups
!gatk AddOrReplaceReadGroups \
    -I {sorted_bam} \
    -O {rg_bam} \
    -RGID 1 -RGLB lib1 -RGPL illumina -RGPU unit1 -RGSM {sample_id}

# ✅ Step 4: Index BAM
!samtools index {rg_bam}

# ✅ Step 5: Check final BAMs
!ls -lh {bam_dir}


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 198020 sequences (20000020 bp)...
[M::process] read 198020 sequences (20000020 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (2, 80449, 1, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (127, 152, 187)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (7, 307)
[M::mem_pestat] mean and std.dev: (158.79, 44.20)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 367)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 198020 reads in 45.020 CPU sec, 29.779 real sec
[M::process] read 198020 sequences (20000020 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 80596, 1, 0)
[M::mem_pestat] skip orientatio

In [19]:
vcf_dir = "/content/drive/MyDrive/Cancer_WES_Project/vcf_files"
bcf_file = f"{vcf_dir}/{sample_id}_raw.bcf"
vcf_file = f"{vcf_dir}/{sample_id}_raw.vcf"

# Generate BCF (binary variant file)
!bcftools mpileup -Ou -f {ref} {rg_bam} | bcftools call -mv -Ob -o {bcf_file}

# Convert BCF to human-readable VCF
!bcftools view {bcf_file} -Ov -o {vcf_file}

# List result
!ls -lh {vcf_dir}


Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
total 78M
-rw------- 1 root root 11M Aug  7 13:55 SRR766039_raw.bcf
-rw------- 1 root root 68M Aug  7 13:55 SRR766039_raw.vcf


In [21]:
import pandas as pd
import io

# Path to your VCF file
vcf_path = "/content/drive/MyDrive/Cancer_WES_Project/vcf_files/SRR766039_raw.vcf"

# Read the file, skipping meta lines (starting with ##)
with open(vcf_path, 'r') as f:
    lines = [line for line in f if not line.startswith('##')]

# Now use io.StringIO instead of pandas.compat
vcf_df = pd.read_csv(io.StringIO(''.join(lines)), sep='\t')

# Show first few rows
vcf_df.head()


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR766039
0,chr1,10150,.,C,T,5.00874,.,DP=4;SGB=-0.379885;RPBZ=1.22474;MQBZ=1.41421;M...,GT:PL,"1/1:30,0,0"
1,chr1,10321,.,C,T,5.04598,.,DP=1;SGB=-0.379885;FS=0;MQ0F=0;AC=2;AN=2;DP4=0...,GT:PL,"1/1:33,3,0"
2,chr1,14653,.,C,T,129.00700,.,DP=55;VDB=0.0024534;SGB=-0.693127;RPBZ=-0.7989...,GT:PL,"1/1:156,11,0"
3,chr1,14677,.,G,A,207.08800,.,DP=78;VDB=0.00065505;SGB=-0.693147;RPBZ=-2.881...,GT:PL,"1/1:234,32,0"
4,chr1,16257,.,G,C,8.88822,.,DP=5;VDB=0.0221621;SGB=-0.511536;RPBZ=-1.73205...,GT:PL,"0/1:41,0,33"


In [22]:
vcf_df.to_csv("/content/drive/MyDrive/Cancer_WES_Project/vcf_files/SRR766039_raw.csv", index=False)


In [23]:
from google.colab import files

# Provide the full path to your CSV file
files.download("/content/drive/MyDrive/Cancer_WES_Project/vcf_files/SRR766039_raw.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
from google.colab import files

# Download VCF (main file for annotation)
files.download("/content/drive/MyDrive/Cancer_WES_Project/vcf_files/SRR766039_raw.vcf")




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>